# Make sure Colab Hardware accelerator is set to :T4 GPU

In [ ]:
import time
start_time = time.time()

In [ ]:
!git clone https://github.com/openai/shap-e.git

fatal: destination path 'shap-e' already exists and is not an empty directory.


In [ ]:
!pip install shap-e/

Processing ./shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-0713ozk1/clip_b0c8e5579b11400b9a5cf090f4ff462a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-0713ozk1/clip_b0c8e5579b11400b9a5cf090f4ff462a
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for shap-e: filename=shap_e-0.0.0-py3-none-any.whl size=133335 sha256=1599f9165cb4b187c60d27fccd3b706546f584d9c8fd214714c23fb5c8eeda57
  Stored in directory: /root/.cache/pip/wheels/66/1e/51/d91f5205d466ed744d67c08e54642f6693baa79bcae83a75dd
Successfully built shap-e
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e-0.0.0


# Text to Shape

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [ ]:
batch_size = 4
guidance_scale = 15.0
prompt = "a green table lamp"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAL/am77Zm73ZmrzYmbvWmbnUl7XVkbTUkbTSkbTPlK/Nja3JjK…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAKbDhqXDhaXChqTChKTBhaPBg6PAhKLAg6O+g6G/g6C+gqS9g6…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAK7PmK/OmK7Oma7Ol63Pl63Plq3OmKzPlqvPlazOlqzOlavOla…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAMXevMjcvsXcvcXcvMfbvsXbvs3ZvsrZvsfavcPcvcDducDcuc…

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


# Image to Shape

In [ ]:
from shap_e.util.image_util import load_image

batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("shap-e/shap_e/examples/example_data/corgi.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPry9Pbv7/Xr6vLb0NfX0fbRu9vPw9DMw+TFqdfFoNLAqdK/m8…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAP38/fz8/fz7/Pv7/Pv6+/r6+/n5+vj3+fb2+PXz9fLw8vPu8e…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAO3r6unp6uXi4urZuvTXiOzQrO/Shu7OgdvNu+fHo97Fm+bHg+…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPf39/X19fX09PDw7+jp6OTn5+Pl5eHi4tzi39vd2tjd29bb29…

In [ ]:
end_time = time.time()
print(f"Time taken: {int(end_time - start_time)} seconds")

Time taken: 350 seconds
